In [1]:
from datasets import load_dataset

cifar_10 = load_dataset('cifar10')

In [2]:
labels = cifar_10['train'].features['label'].names

In [3]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel
import numpy as np

In [4]:
model_path = "/data-ai/usr/lmj/models/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_path)
processor = AutoProcessor.from_pretrained(model_path)

In [78]:
from tqdm import trange

# get image feature
def get_sample(partition: str):
    num = len(cifar_10[partition])
    batch_size = 20
    images, label_ids = np.empty(shape=(num, 512), dtype=np.float32), np.empty(shape=(num, 1), dtype=np.int8)
    data = cifar_10[partition]
    for n in trange(0, num, batch_size):
        batch_images, batch_label_ids = data[n:n+batch_size]['img'], [[_] for _ in data[n:n+batch_size]['label']]
        label_ids[n:n+batch_size, :] = batch_label_ids
        inputs = processor(images=batch_images, return_tensors="pt")
        image_features = model.get_image_features(**inputs).detach().numpy()
        images[n:n+batch_size, :] = image_features
    return images, label_ids.ravel()

In [79]:
train_images, train_labels = get_sample('train')

100%|██████████| 2500/2500 [09:46<00:00,  4.26it/s]


In [80]:
test_images, test_labels = get_sample('test')

100%|██████████| 500/500 [01:57<00:00,  4.26it/s]


In [81]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, random_state=0, C=0.316).fit(train_images, train_labels)

/opt/conda/envs/lmj/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [82]:
pred_result = clf.predict(test_images)

In [83]:
from sklearn.metrics import classification_report

print(classification_report(test_labels, pred_result, target_names=labels, digits=4))

              precision    recall  f1-score   support

    airplane     0.9631    0.9660    0.9646      1000
  automobile     0.9750    0.9760    0.9755      1000
        bird     0.9412    0.9280    0.9345      1000
         cat     0.8924    0.9040    0.8982      1000
        deer     0.9266    0.9340    0.9303      1000
         dog     0.9291    0.9170    0.9230      1000
        frog     0.9467    0.9600    0.9533      1000
       horse     0.9757    0.9630    0.9693      1000
        ship     0.9770    0.9780    0.9775      1000
       truck     0.9750    0.9750    0.9750      1000

    accuracy                         0.9501     10000
   macro avg     0.9502    0.9501    0.9501     10000
weighted avg     0.9502    0.9501    0.9501     10000

